# Relativistic 2-body simulation

Version history

In [ ]:
# @title
# 2025-04-21  2-body-com v5.jpynb
# Restructured code for better readability in Jupyter Notebook

# 2025-01-09  2-body-com v4.jpynb
# Converted code to Jupyter Notebook

# 2024-08-17  2-body-com v4.py
# Added animated illustration of GW radiation

# 2024-08-01  2-body-com v3.py
# Implemented constellation as in Kidder Fig 4, based on method to determine initial velocity for a circular orbit
# Added calculation of GW radiation far field (perturbation tensor h, projection to h+/hx). Results directionally aligned with Kidder Fig 4
# More precise calculation of body positions, according to formulas by Blanchet
# Added GW near field calculation. Results directionally aligned with far field

# 2024-08-01  2-body-com v2.py
# Implemented dL/dt analytics. Kidder paper provides faulty formula, applied guesswork to fix formula (fixes match numerical results)
# Implemented dP/dt analytics
# Introduced termination of solve_ivp when distance comes closer than 1*M
# Added PSR J1757-1854. Simulation matches all relevant parameters and results in correct periastron shift and orbital period change
# Spin precession is at 1.3 deg/year, compared to expected value of 3 deg/year. Reason unclear.

# 2024-07-29  2-body-com v1.py
# Changed to center of mass frame. Leads to simplified equations for acceleration, conserved quantities and spin precession.
# Implemented analytical loss of energy by radiation calculation
# Improved periastron shift calculation to also work outside equatorial plane

# 2024-07-25  2-body v6.py
# Introduced Hulse-Taylor Pulsar (PSR 1913 + 16). Added calculation of periastron points and determination of periastron shift.
# Simulation leads correct periastron shift (ca 13 arcsec per orbit)
# Amended initial position for the 2 yukterez.com examples to better match true orbits
# Re-introduced Mercury orbit. Simulation leads correct perihelion shift (ca 0.1 arcsec per orbit)

# 2024-07-23  2-body v5.py
# Introduced spins S1 and S2 as part of the differential euqation to prepare spin precession
# Introduced Spin Precession modelling. Inspiralling binaries example leads to plausible results (angular momentum pretty constant in magnitude and direction)
# Extension of chart to show spins and angular momentum vectors

# 2024-07-20  2-body v4.py
# Fixed an error in the (very long) 3PN acceleration term. But the 3PN term still leads to a less good match for the yuktaraz.net orbits
# (prograde and retrograde). Not clear why.
# Introduced calculation of conserved quantity Energy (E) to Newtonian, 1PN, 1.5PN (Spin-Orbit) and 2PN order. There are still some variations in E
# Introduced calculation of conserved quantity Impulse (P) and Angular Momentum to Newtonian, 1PN, and 2PN order. Angular momentum is not conserved yet
# as spin precession is not implemented

# 2024-07-20  2-body v3.py
# Extended to 3 dimensions (added z)
# First implementation of 3PN correction (RESULTS ARE NOT PLAUSIBLE, BUT CODE REMAINS IN FOR LATER IMPROVEMENT)
# Correction of a sign issue - now orbits are plausible
# Adoption of yukterez orbits (prograde and retrograde). In case of spin=0, orbits match those of 1 body simulation closely
# when initial velocity is scaled down by a factor of ca. 0.7
# Introduction of leading Spin-Orbit PN terms at 1.5PN level. Orbits match those of 1 body simulation reasonably well

# 2024-07-16  2-body v2.py
# Introduced animated 3d-chart

# 2024-07-15  2-body v2.py
# Introduced 2pn. No difference to 1pn visible for configurations 1) and 2), slight difference for 3)
# Introduced 2.5pn. No difference to 2pn visible for configurations 1) and 2), but 3) shows very strong
# inspiralling effect (degree consistent with expectations)

# 2024-07-14  2-body v1.py
# Newton plus 1PN working with 3 configurations: 1) Earth-Sun, 2) S2-SgtA*, 3) Inspiralling binaries

Imports

In [ ]:
# @title
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d import Axes3D  # This is necessary for 3D plotting
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

G=c=m_sun=year=spin1=spin2=S1n0=S2n0=count=plotOrbit=analyzePeriastron=analyzeDeflection=analyzeRadiation=analyzePerturbation=scientific=r_terminal=rtol=res=name=approximation=N=dist = None
m=dm=my=ny=r_scale=t_scale=r0_nat = None

Helper functions

In [ ]:
# @title
def initialize_globals():
    # Constants and variables
    global G,c,m_sun,year,spin1,spin2,S1n0,S2n0,count,plotOrbit,analyzePeriastron,analyzeDeflection,analyzePerturbation,analyzeRadiation,scientific,r_terminal,rtol,res,name,approximation,N,dist
    global m,dm,my,ny,r_scale,t_scale,r0_nat

    G = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-2
    c = 299792458    # Speed of light, m/s
    m_sun = 1.989e30  # Mass of the Sun, kg
    year = 365.25*24*3600 # year in s
    spin1 = spin2 = 0 # Kerr metric's a parameter
    S1n0 = S2n0 = np.array([0, 0, 1])
    count = 0
    plotOrbit = True
    analyzePeriastron = False
    analyzeDeflection = False
    analyzeRadiation = False
    analyzePerturbation = False
    scientific = False # Style of plotly charts
    r_terminal = 1
    rtol = 1e-9
    res = 10000 # number of data points
    name = "Orbit"
    approximation = 3.0 # default approximation order
    N = np.array([1, 0, 0]) # direction of observer
    dist = 10 * c # observer distance
    return

def s_outer(a,b):
    return 0.5*np.outer(a,b) + 0.5*np.outer(b,a)

def v_circular(LNn): # detemines velocity for quasi circular orbit
    m = m1+m2
    ny = m1*m2/m**2
    r_omega_sq = m/r0*( 1-(3-ny)*m/r0 - ( spin1*np.dot(LNn,S1n0)*(2*m1**2/m**2+3*ny) + spin2*np.dot(LNn,S2n0)*(2*m2**2/m**2+3*ny) )*(m/r0)**(3/2) + ( (6+41/4*ny+ny**2) - 3/2*ny*spin1*spin2*(np.dot(S1n0,S2n0) - 3*np.dot(LNn,S1n0)*np.dot(LNn,S2n0) ) )*(m/r0)**2 )
    return np.sqrt(r_omega_sq)

def polarization_tensors(ex, ey):
    E_plus = np.outer(ex, ex) - np.outer(ey, ey)
    E_cross = np.outer(ex, ey) + np.outer(ey, ex)
    return E_plus, E_cross

def project_onto_basis(X, E_plus, E_cross):
    h_plus = 0.5*np.tensordot(X, E_plus, axes=2)
    h_cross = 0.5*np.tensordot(X, E_cross, axes=2)
    return h_plus, h_cross

def omegaZAMO(r, x, rs, spin, theta, vtheta, vphi):     # ZAMO correction
    vlocal = np.sqrt((vtheta*r)**2 + (vphi*x)**2)
    lorenzV = 1/np.sqrt(1 - vlocal**2)
    sigma_zamo = r**2 + spin**2 * np.cos(theta)**2
    delta_zamo = r**2 - rs*r + spin**2
    xi_zamo = (spin**2 + r**2)**2 - spin**2 * np.sin(theta)**2 * delta_zamo
    omega_zamo = rs*spin1*r/xi_zamo # ZAMO angular velocity in 1/s
    alpha = np.sqrt(xi_zamo/delta_zamo/sigma_zamo) # Gravitational red shift
    return omega_zamo*alpha # adjust local velocity with omega

def conservedQuantities(y):
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz])
    v = np.array([vx, vy, vz])
    S1 = np.array([S1x, S1y, S1z])
    S2 = np.array([S2x, S2y, S2z])
    r = np.linalg.norm(x)
    n = x / r
    S = S1 + S2 # kg^2
    D = D_func(S1, S2)
    v_sq = np.dot(v, v)
    L_N = my*np.cross(x, v) # kg^2
    dr = np.dot(x, v)/r # 1

    # Newton
    E = my*(1/2*v_sq - m/r) # kg
    J = L_N # kg^2
    dE = -8/15*(m*my/r**2)**2 * ( 12*v_sq - 11*dr**2 ) # 1
    dJ = -8/5*m*my/(r**3)*L_N * ( 2*v_sq - 3*dr**2 + 2*m/r ) # kg KIDDER PROVIDES FAULTY FORMULA
    dP = -108/5*dm/m*(ny**2)*(m/r)**4 * ( dr*n*(55*v_sq-45*dr**2+12*m/r) + v*(38*dr**2-50*v_sq-8*m/r) ) # 1

    if approximation >= 1: # 1PN
        E += my*( 3/8*(1-3*ny)*(v_sq**2) + 1/2*(3+ny)*v_sq*m/r + 1/2*ny*m/r*(dr**2) + 1/2*(m/r)**2 ) #kg
        J += L_N*( 1/2*v_sq*(1-3*ny) + (3+ny)*m/r ) # kg^2
        dE += -2/105*(m*my/r**2)**2 * ( (785-852*ny)*v_sq**2 - 160*(17-ny)*m/r*v_sq + 8*(367-15*ny)*m/r*dr**2 - 2*(1487-1392*ny)*v_sq*dr**2 + 3*(687-620*ny)*dr**4 + 16*(1-4*ny)*(m/r)**2 )
        dJ += -2/105*m*my/(r**3)*L_N * ( (307-548*ny)*v_sq**2 - 6*(74-277*ny)*v_sq*dr**2 + 2*(372+197*ny)*m/r*dr**2 + 15*(19-72*ny)*dr**4 - 4*(58+95*ny)*m/r*v_sq - 2*(745-2*ny)*(m/r)**2 )
    if approximation >= 1.5: # 1.5PN Spin-Orbit
        E += 1/(r**3)*np.dot(L_N,S+dm/m*D) # kg
        J += my/m*( m/r*np.cross(n,np.cross(n,3*S+dm/m*D)) - 1/2*np.cross(v,np.cross(v,S+dm/m*D)) )
        dE += -8/15*m*my/(r**6) * np.dot(L_N, S*(78*dr**2 - 80*v_sq - 8*m/r) + dm/m*D*(51*dr**2 - 43*v_sq + 4*m/r) )
        # dJ += xxx.  NOT YET IMPLEMENTED
        dP += -8/15*(my**2)*m/(r**5) * ( 4*dr*np.cross(v,D) - 2*v_sq*np.cross(n,D) - np.cross(n,v)*( 3*dr*np.dot(n,D)+2*np.dot(v,D) ) )
    if approximation >= 2: # 2PN
        E += my*( 5/16*(1-7*ny+13*ny**2)*v_sq**3 - 3/8*ny*(1-3*ny)*m/r*dr**4 + 1/8*(21-23*ny-27*ny**2)*m/r*v_sq**2 + 1/8*(14-55*ny+4*ny**2)*(m/r)**2*v_sq + 1/4*ny*(1-15*ny)*m/r*v_sq*dr**2 - 1/4*(2+15*ny)*(m/r)**3 + 1/8*(4+69*ny+12*ny**2)*(m/r)**2*dr**2 )
        J += L_N*( 3/8*(1-7*ny+13*ny**2)*v_sq**2 - 1/2*ny*(2+5*ny)*m/r*dr**2 + 1/2*(7-10*ny-9*ny**2)*m/r*v_sq + 1/4*(14-41*ny+4*ny**2)*(m/r)**2 )
    if approximation >= 2: # 2PN Spin-Spin
        E += 1/(r**3)*( 3*np.dot(n,S1)*np.dot(n,S2) - np.dot(S1,S2) )
        J += 0 # there is no spin spin component
        dE += -4/15*m*my/(r**6) * ( -3*np.dot(n,S1)*np.dot(n,S2)*(168*v_sq-269*dr**2) + 3*np.dot(S1,S2)*(47*v_sq-55*dr**2) + 71*np.dot(v,S1)*np.dot(v,S2) - 171*dr*( np.dot(v,S1)*np.dot(n,S2)+np.dot(n,S1)*np.dot(v,S2) ) )

    return [E*float(c**2), (J + S)*G/c, dE/G*float(c**5), dJ*float(c**2), dP/G*float(c**4)] # [kg m^2 s^-2, kg m^2 s^-1, kg m^2 s^-3, kg m^2 s^-2, kg m s^-2]

def farField(y, dist_nat, N): # returns far field tensor
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz])
    v = np.array([vx, vy, vz])
    S1 = np.array([S1x, S1y, S1z])
    S2 = np.array([S2x, S2y, S2z])
    r = np.linalg.norm(x)
    n = x / r
    S = S1 + S2 # kg^2
    D = D_func(S1, S2)
    v_sq = np.dot(v, v)
    L_N = my*np.cross(x, v) # kg^2
    dr = np.dot(x, v)/r # 1
    v_so_v = np.outer(v,v)
    n_so_n = np.outer(n,n)
    n_so_v = s_outer(n,v)

    #h = 0*v_so_v
    h = 2*(v_so_v - m/r*n_so_n) # Quadrupol component (Kidder 3.22a)
    if approximation >= 0.5: # 0.5PN (Kidder 1 formula 3.22b)
        h += dm/m*( 3*m/r*np.dot(N,n)*(2*n_so_v-dr*n_so_n) + np.dot(N,v)*(m/r*n_so_n-2*v_so_v) )
    if approximation >= 1:
        # 1PN (Kidder 1 formula 3.22c)
        h += 1/3*(1-3*ny)*( 4*m/r*(3*dr*n_so_n-8*n_so_v)*np.dot(N,n)*np.dot(N,v) + 2*(3*v_so_v-m/r*n_so_n)*np.dot(N,v)**2 + m/r*( (3*v_sq-15*dr**2+7*m/r)*n_so_n+30*dr*n_so_v-14*v_so_v)*np.dot(N,n)**2 ) + 4/3*m/r*dr*(5+3*ny)*n_so_v + ( (1-3*ny)*v_sq-2/3*(2-3*ny)*m/r)*v_so_v + m/r*( (1-3*ny)*dr**2-1/3*(10+3*ny)*v_sq+29/3*m/r)*n_so_n
        # 1PN Spin-Orbit (Kidder 1 formula 3.22d)
        D_cross_N = np.cross(D,N)
        h += 2/r**2*( 0.5*np.outer(D_cross_N,n) + 0.5*np.outer(n,D_cross_N) )
    if approximation >= 1.5:
        # 1.5PN (Kidder 1 formula 3.22e)
        h += dm/m*(1-2*ny)*( 1/4*m/r*( (45*dr**2-9*v_sq-28*m/r)*n_so_n + 58*v_so_v - 108*dr*n_so_v )*np.dot(N,n)**2*np.dot(N,v) + 1/2*(m/r*n_so_n-4*v_so_v )*np.dot(N,v)**3 + m/r*(5/4*(3*v_sq-7*dr**2+6*m/r)*dr*n_so_n - 1/6*(21*v_sq-105*dr**2+44*m/r)*n_so_v - 17/2*dr*v_so_v  )*np.dot(N,n)**3 + 3/2*m/r*(10*n_so_v-3*dr*n_so_n  )*np.dot(N,n)*np.dot(N,v)**2 )
        h += 1/12*dm/r*np.dot(N,n)*( n_so_n*dr*(dr**2*(15-90*ny)-v_sq*(63-54*ny)+m/r*(242-24*ny)) - dr*v_so_v*(186+24*ny) + 2*n_so_v*(dr**2*(63+54*ny)-m/r*(128-36*ny)+v_sq*(33-18*ny)) )
        h += dm/m*np.dot(N,v)*( 1/2*v_so_v*( m/r*(3-8*ny)-2*v_sq*(1-5*ny) ) - n_so_v*m/r*dr*(7+4*ny) - n_so_n*m/r*(3/4*(1-2*ny)*dr**2 + 1/3*(26-3*ny)*m/r - 1/4*(7-2*ny)*v_sq ) )
        # 1.5PN Spin-Orbit (Kidder 1 formula 3.22f)
    if approximation >= 2: # 2PN Spin-Spin
        h += -6/my*r**3*( 0 ) # NOT COMPLETE
    return 2*my/dist_nat*h # 1

def nearField(y, dist_nat, N): # returns near field tensor
    def nearFieldComponent(ma, mb, r, ra, rb, va, vb, na, nb, n, v):
        # BLANCHET 7 FORMULA 7.2c
        S = ra + rb + r
        h  = 2*ma/ra*np.eye(3)
        coeff = -ma/ra*np.dot(na,va)**2 + (ma/ra)**2 + ma*mb*( 2/ra/rb - ra/2/r**3 + ra**2/2/rb/r**3 - 5/2/ra/r + 4/r/S)
        h += np.eye(3)*coeff
        h += 4*ma/ra*np.outer(va,va)
        h += (ma/ra)**2 * np.outer(na,na) - 4*ma*mb*np.outer(n,n)*(1/S**2+1/r/S) + 4*ma*mb/S**2*(s_outer(na,nb)+2*s_outer(na,n) )
        #h += ma*mb/r**2 * ( -2/3*np.dot(n,v)*np.eye(3) - 6*np.dot(n,v)*np.outer(n,n) + 8*s_outer(n,v) ) # THIS TERM IS NOT PLAUSIBLE - OBSERVER DISTANCE IRRELEVANT TO THE TERM
        #h += ma*mb/ra**2 * ( -2/3*np.dot(n,v)*np.eye(3) - 6*np.dot(n,v)*np.outer(n,n) + 8*s_outer(n,v) ) # WILD GUESS!!!!!!
        return h

    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz]) # position body 1 - body 2
    v = np.array([vx, vy, vz]) # same for velocity
    r = np.linalg.norm(x) # same for distance
    n = x / r #
    x1, x2 = bodyPositions(y) # body positions versus com
    r1 = np.linalg.norm(dist_nat*N - x1) # distance observer to body1
    r2 = np.linalg.norm(dist_nat*N - x2) # distance observer to body2
    n1 = (dist_nat*N - x1)/r1 # direction body 1 to observer
    n2 = (dist_nat*N - x2)/r2 # direction body 2 to observer
    v1 =  np.linalg.norm(x1)/r * v # body 1 velocity
    v2 = -np.linalg.norm(x2)/r * v # body 2 velocity
    n12 =  n
    n21 = -n
    v12 =  v
    v21 = -v

    h  = nearFieldComponent(m1, m2, r, r1, r2, v1, v2, n1, n2, n12, v12)
    h += nearFieldComponent(m2, m1, r, r2, r1, v2, v1, n2, n1, n21, v21)
    return h # 1

def bodyPositions(y): # returns the position of both bodies in the com frame
    # Formulas: Blanchet, emis.de, chapter 7.3
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz])
    v = np.array([vx, vy, vz])
    r = np.linalg.norm(x)
    n = x / r
    v_sq = np.dot(v, v)
    dr = np.dot(x, v)/r # 1
    X1 = m1/m
    X2 = m2/m
    D = X1 - X2
    P = 1/2*(v_sq-m/r) + ( 3/8*v_sq**2-3/2*ny*v_sq**2 + m/r*(-1/8*dr**2+3/4*dr**2*ny+19/8*v_sq+3/2*ny*v_sq) + (m/r)**2*(7/4-ny/2) )
    Q = -7/4*m*dr + 4/5*m*v_sq - 8/5*m**2/r
    return [(X2+ny*D*P)*x + ny*D*Q*v, (-X1+ny*D*P)*x + ny*D*Q*v]

def distance(y): # distance in m
    return distance_nat(y)*r_scale # m

def distance_nat(y): # distance in kg
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz])
    return np.linalg.norm(x) # kg

def angle(y_prior, y): # determines the angle (in deg) between 2 states, as defined by their x vector
    x_prior = np.array([y_prior[0], y_prior[1], y_prior[2]])
    x = np.array([y[0], y[1], y[2]])
    cosPhi = np.dot(x_prior, x)/np.linalg.norm(x)/np.linalg.norm(x_prior)
    return np.arccos(cosPhi)/np.pi*180 # deg

def dS_1pn(r, x, ma, mb, n, v, Sa, Sb): # leading Spin Precession (Spin-Orbit) PN term
    L_N = my*np.cross(x, v) # kg^2
    if ma>0: return 1/(r**3)*( (2+3/2*mb/ma)*np.cross(L_N,Sa) - np.cross(Sb,Sa) + 3*np.dot(n,Sb)*np.cross(n,Sa) ) # kg
    else: return np.zeros(3)

def D_func(S1, S2):
    if my>0: return m*(S2/m2 - S1/m1) # kg^2
    else: return S2-S1

def acceleration(t, y):
    global count
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y # Unpack state vector
    x = np.array([xx, xy, xz]) # kg
    v = np.array([vx, vy, vz]) # 1
    S1 = np.array([S1x, S1y, S1z]) # kg^2
    S2 = np.array([S2x, S2y, S2z]) # kg^2
    r = np.linalg.norm(x) # kg
    n = x / r # 1
    v_sq = np.dot(v, v) # 1
    dr = np.dot(n, v) # 1
    S = S1 + S2 # kg^2
    D = D_func(S1, S2)

    # Compute Newtonian gravitational forces
    a = -m/(r**2) * n # kg^-1

    # 1PN corrections
    if approximation >= 1:
        # Kidder
        coeff_n = (1 + 3*ny)*v_sq - 2*(2+ny)*m/r - 3/2*ny*(dr**2) # 1
        coeff_v = -2*(2 - ny)*dr #1
        a += -m/(r**2)*(coeff_n * n + coeff_v * v) # kg^-1
        """
        # Forumla according to Blanchet on emis.de (result in agreement with Kidder)
        a += -m/(r**2)*( n*(-3/2*dr**2*ny + v_sq + 3*ny*v_sq -m/r*(4+2*ny)) + v*(-4*dr+2*dr*ny) )
        """

    # 1.5PN Spin-Orbit corrections
    if approximation >= 1.5:
        a += 1/(r**3)*( 6*n*np.dot(np.cross(n,v), 2*S+dm/m*D) - np.cross(v, 7*S+3*dm/m*D) + 3*dr*np.cross(n, 3*S+dm/m*D) )

    # 2PN corrections
    if approximation >= 2:
        # Kidder
        coeff_n = 3/4*(12+29*ny)*(m/r)**2 + ny*(3-4*ny)*v_sq**2 + 15/8*ny*(1-3*ny)*dr**4 - 3/2*ny*(3-4*ny)*v_sq*dr**2 - 1/2*ny*(13-4*ny)*m/r*v_sq - (2+25*ny+2*ny**2)*m/r*dr**2
        coeff_v = ny*(15+4*ny)*v_sq - (4+41*ny+8*ny**2)*m/r - 3*ny*(3+2*ny)*dr**2
        a += -m/(r**2)*(coeff_n * n - 1/2*dr*coeff_v * v)
        """
        # Forumla according to Blanchet on emis.de (result in agreement with Kidder)
        coeff_n = 15/8*dr**4*ny - 45/8*dr**4*ny**2 - 9/2*dr**2*ny*v_sq + 6*dr**2*ny**2*v_sq + 3*ny*v_sq**2 - 4*ny**2*v_sq**2 + m/r*(-2*dr**2-25*dr**2*ny-2*dr**2*ny**2-13/2*ny*v_sq+2*ny**2*v_sq) + (m/r)**2*(9+87/4*ny)
        coeff_v = 9/2*dr**3*ny + 3*dr**3*ny**2 - 15/2*dr*ny*v_sq - 2*dr*ny**2*v_sq + m/r*(2*dr+41/2*dr*ny+4*dr*ny**2)
        a += -m/(r**2)*( n*coeff_n + v*coeff_v )
        """

    # Spin-Spin correction
    if approximation >= 2:
        if my>0: a += -3/my/(r**4)*( n*np.dot(S1,S2) + S1*np.dot(n,S2) + S2*np.dot(n,S1) - 5*n*np.dot(n,S1)*np.dot(n,S2) )

    # 2.5PN corrections (Radiation)
    if approximation >= 2.5:
        # Kidder
        coeff_n = 18*v_sq + 2/3*m/r - 25*dr**2
        coeff_v = 6*v_sq - 2*m/r - 15*dr**2
        a += 8/5*ny*(m**2)/(r**3)*( dr*coeff_n * n - coeff_v * v )
        """
        # Forumla according to Blanchet on emis.de (RESULT NOT IN AGREEMENT WITH KIDDER)
        coeff_n = 3*v_sq + 17/3*(m/r)
        coeff_v = v_sq + 3*(m/r)
        a += 8/5*ny*m**2/(r**3)*( dr*n*coeff_n - v*coeff_v )
        """

    # 3PN corrections (source: Blanchet. No formula provided by Kidder)
    if approximation >= 3:
        """
        coeff_n  = -35/16*dr**6*ny + 175/16*dr**6*ny**2 - 175/16*dr**6*ny**3 + 15/2*dr**4*ny*v_sq - 135/4*dr**4*ny**2*v_sq + 255/8*dr**4*ny**3*v_sq - 15/2*dr**2*ny*v_sq**2 + 237/8*dr**2*ny**2*v_sq**2 - 45/2*dr**2*ny**3*v_sq**2 + 11/4*ny*v_sq**3 - 49/4*ny**2*v_sq**3 + 13*ny**3*v_sq**3
        coeff_n += m/r*(79*dr**4*ny - 69/2*dr**4*ny**2 - 30*dr**4*ny**3 - 121*dr**2*ny*v_sq + 16*dr**2*ny**2*v_sq + 20*dr**2*ny**3*v_sq + 75/4*ny*v_sq**2 + 8*ny**2*v_sq**2 - 10*ny**3*v_sq**2)
        coeff_n += (m/r)**2*(dr**2 + 32573/168*dr**2*ny + 11/8*dr**2*ny**2 - 7*dr**2*ny**3 + 615/64*dr**2*ny*np.pi**2 - 26987/840*ny*v_sq + ny**3*v_sq - 123/64*ny*np.pi**2*v_sq - 110*dr**2*ny*np.log(r/r0_nat) + 22*ny*v_sq*np.log(r/r0_nat))
        coeff_n += (m/r)**3*(-16 - 437/4*ny - 71/2*ny**2 + 41/16*ny*np.pi**2)
        coeff_v  = -45/8*dr**5*ny + 15*dr**5*ny**2 + 15/4*dr**5*ny**3 + 12*dr**3*ny*v_sq - 111/4*dr**3*ny**2*v_sq - 12*dr**3*ny**3*v_sq - 65/8*dr*ny*v_sq**2 + 19*dr*ny**2*v_sq**2 + 6*dr*ny**3*v_sq**2
        coeff_v += m/r*(329/6*dr**3*ny + 59/2*dr**3*ny**2 + 18*dr**3*ny**3 - 15*dr*ny*v_sq - 27*dr*ny**2*v_sq - 10*dr*ny**3*v_sq)
        coeff_v += (m/r)**2*(-4*dr - 18169/840*dr*ny + 25*dr*ny**2 + 8*dr*ny**3 - 123/32*dr*ny*np.pi**2 + 44*dr*ny*np.log(r/r0_nat))
        a += -m/(r**2)*( n*coeff_n + v*coeff_v )
        """

    # Spin precession
    dS1 = dS2 = np.zeros(3)
    if approximation >= 1:
        dS1 = dS_1pn(r, x, m1, m2, n, v, S1, S2) # kg
        dS2 = dS_1pn(r, x, m2, m1, n, v, S2, S1)

    count = count + 1
    return [vx, vy, vz, a[0], a[1], a[2], dS1[0], dS1[1], dS1[2], dS2[0], dS2[1], dS2[2]]

def accNorm(t, y):
    acc = acceleration(t, y)
    return np.linalg.norm(acc[3:6])/G*c**4

def dr_func(t, y): # for periastron detection
    xx, xy, xz, vx, vy, vz, S1x, S1y, S1z, S2x, S2y, S2z = y
    x = np.array([xx, xy, xz]) # kg
    v = np.array([vx, vy, vz]) # 1
    r = np.linalg.norm(x) # kg
    return np.dot(x, v)/r # 1

def terminate_func(t, y): # terminate solve_ivp
    return distance_nat(y)/m - r_terminal # r_terminal=1 means termination at 0.5 Schwarzschild radius

terminate_func.terminal = True  # Terminate integration at the event
terminate_func.direction = -1  # Trigger on negative second derivative (falling below the minimum distance)

dr_func.terminal = False  # Do not terminate integration at the event
dr_func.direction = +1  # Trigger on positive second derivative of radial velocity (approaching periastron)

Main function to simulate orbit

In [ ]:
# @title
def simulate_orbit():
    global G,c,m_sun,year,spin1,spin2,S1n0,S2n0,count,plotOrbit,analyzePeriastron,analyzeDeflection,analyzePerturbation,analyzeRadiation,scientific,r_terminal,rtol,res,name,approximation,N,dist
    global m,dm,my,ny,r_scale,t_scale,r0_nat

    # Combine initial conditions into a single array
    m = m1 + m2 # kg
    dm = m1 - m2 # kg
    my = m1*m2/m # kg
    ny = my/m # 1
    r_scale = G/c**2 # m/kg
    t_scale = r_scale/c # s/kg
    N = N/np.linalg.norm(N)
    r0_nat = r0/r_scale

    S1_initial = S1n0 * (m1**2) * spin1 # kg^2
    S2_initial = S2n0 * (m2**2) * spin2 # kg^2
    y0 = np.concatenate((-r_initial/r_scale, -v_initial/c, S1_initial, S2_initial)) # [kg, 1, kg^2, kg^2]
    cQ0 = conservedQuantities(y0) # [E,J,dE,dJ,dP]
    normD0 = np.linalg.norm(r_initial) # m
    normE0 = abs(cQ0[0]) # J
    normJ0 = np.linalg.norm(cQ0[1]) # kg m^2 s^-1
    normS10 = np.linalg.norm(S1_initial) # kg^2
    normS20 = np.linalg.norm(S2_initial) # kg^2
    if normE0 < 1e-100: normE0 = 1
    if normJ0 < 1e-100: normJ0 = 1
    if normS10 < 1e-100: normS10 = 1
    if normS20 < 1e-100: normS20 = 1

    # Time points where the solution is computed
    dt = t_span[1] / res # s
    t_nat_span = [t_span[0]/t_scale, t_span[1]/t_scale] # kg
    t_nat_eval = np.linspace(t_nat_span[0], t_nat_span[1], res) # kg

    print(name)
    print("Initial situation:")
    print(f"  m1/2={m1:.3e}/{m2:.3e}kg, r0={normD0:.3e}m, |S1/2|={normS10*G/c:.3e} / {normS20*G/c:.3e}kg*m^2/s, E0={cQ0[0]:.3e}J, |J0|={normJ0:.3e}kg*m^2/s, |a|={accNorm(t_nat_span[0], y0)/9.81:.3e}g")

    print("Natural units:")
    print(f"  r_scale={r_scale:.3e}kg/m, t_scale={t_scale:.3e}kg/s")
    print(f"  r0_nat={np.linalg.norm(r_initial)/r_scale:.3e}kg, r0_nat/m={np.linalg.norm(r_initial)/r_scale/m:.3e}, v0_nat={np.linalg.norm(v_initial)/c:.3e}, t1_nat={t_nat_span[1]:.3e}kg")

    # Solve the differential equations with higher accuracy
    sol = solve_ivp(acceleration, t_nat_span, y0, t_eval=t_nat_eval, events=[dr_func, terminate_func], method='DOP853', rtol=rtol, atol=1e-8*rtol)
    nn = len(sol.y[0])
    periastron_times = sol.t_events[0]
    periastron_states = sol.y_events[0]
    print("Differenial Equation solution:")
    print(f"  Data points: {len(sol.y[0])}, #acceleration calls={count}")
    print(f"Periastron points: {len(periastron_states)}")
    per_n = len(periastron_states)
    if analyzePeriastron:
        timeFirstToLast = (periastron_times[per_n-1] - periastron_times[0])*t_scale
        deltaPhi = 0 # initialize deltaPhi (deg)
        for k in range(per_n):
            per_cQ = conservedQuantities(periastron_states[k])
            per_r = distance(periastron_states[k])
            if k==0: # memorize first periastron as a reference
                first_per_cQ = per_cQ
                first_per_r = per_r
            if k>0: # determine changes of orbital period
                deltaPhi += angle(periastron_states[k-1], periastron_states[k])
                orbitPeriod = (periastron_times[k]-periastron_times[k-1])*t_scale
                if k==1: firstOrbitPeriod = orbitPeriod # memorize first orbital period
                if deltaPhi > 1:    print(f"  {k}: deltaPhi={deltaPhi:.1f} deg")
                elif deltaPhi*60 > 1: print(f"  {k}: deltaPhi={deltaPhi*60:.1f} arcmin")
                else:                 print(f"  {k}: deltaPhi={deltaPhi*3600:.1f} arcsec")
        if k>1: # at least 2 periastron points detected
            deltaOrbitPeriod = orbitPeriod-firstOrbitPeriod
            deltaE = per_cQ[0] - first_per_cQ[0]
            deltaL = per_cQ[1] - first_per_cQ[1]
            deltaLnorm = np.linalg.norm(per_cQ[1]) - np.linalg.norm(first_per_cQ[1])
            deltaR = per_r - first_per_r
            print("Numerical results (between first and last periastron):")
            print(f"  GW deltaE={deltaE:.3e} J in {timeFirstToLast:.3e} s, dE/dt={deltaE/timeFirstToLast:.3e} W. deltaE/E0={deltaE/normE0:.3e} ")
            print(f"  GW deltaL={deltaL[0]:.3e}/{deltaL[1]:.3e}/{deltaL[2]:.3e} kg m^2 s^-1, |J1|-|J0|={deltaLnorm:.3e} kg m^2 s^-1, dL/dt={deltaL[0]/timeFirstToLast:.3e}/{deltaL[1]/timeFirstToLast:.3e}/{deltaL[2]/timeFirstToLast:.3e} kg m^2 s^-2")
            print(f"  Orbit decay dT={deltaOrbitPeriod:.3e} s in {timeFirstToLast:.3e} s, {deltaOrbitPeriod/timeFirstToLast*year:.3e} s/year")
            print(f"  Orbit decay deltaR={deltaR:.3e} m in {timeFirstToLast:.3e} s, dr={deltaR/timeFirstToLast*year:.3e} m/year. deltaR/r0={deltaR/r0:.3e}")

    if analyzeDeflection: # Deflection analysis
        if per_n == 1:  # Check if there is only one periastron
            n_div10 = int(nn/10)
            d_start = sol.y[0:3,n_div10] - sol.y[0:3,0]
            d_end = sol.y[0:3,-1] - sol.y[0:3,-1-n_div10]
            d_start /= np.linalg.norm(d_start)
            d_end /= np.linalg.norm(d_end)
            deflection = np.arccos(np.dot(d_start, d_end))/np.pi*180*3600
            print(f"Deflection={deflection:.3f} arcsec, b={distance(periastron_states[0]):.3e}")

    # Calculate approx. coordinates of both bodies
    bP = [bodyPositions(sol.y[:, k]) for k in range(len(sol.y[0]))]
    r1 = [ [bP[k][0][0] for k in range(nn)], [bP[k][0][1] for k in range(nn)], [bP[k][0][2] for k in range(nn)] ]
    r2 = [ [bP[k][1][0] for k in range(nn)], [bP[k][1][1] for k in range(nn)], [bP[k][1][2] for k in range(nn)] ]
    #r1 = m2/m*sol.y[0:3]*r_scale  # simplified
    #r2 = -m1/m*sol.y[0:3]*r_scale # simplified
    S1 = sol.y[6:9]
    S2 = sol.y[9:12]
    lenS1 = [np.linalg.norm(S1[:, k])/normS10 for k in range(len(S1[0]))]
    lenS2 = [np.linalg.norm(S2[:, k])/normS20 for k in range(len(S2[0]))]
    cQ = [conservedQuantities(sol.y[:, k]) for k in range(len(sol.y[0]))]
    deltaE = 0
    deltaL = np.zeros(3)
    deltaP = np.zeros(3)
    deltaD = np.zeros(3)
    for k in range(len(cQ)):
        deltaE += cQ[k][2] * dt # J
        deltaL += cQ[k][3] * dt # kg m^2 s^-1
        deltaP += cQ[k][4] * dt # kg m s^-1
        deltaD += deltaP * dt # m kg m
    deltaLnorm = np.linalg.norm(cQ0[1]+deltaL) - np.linalg.norm(cQ0[1])
    deltaD /= m # m
    h = [[0,0] for k in range(len(sol.y[0]))]
    print("Analytical results (between start and end of simulation):")
    print(f"  deltaE={deltaE:.4e} J, av. dE/dt={deltaE/t_span[1]:.4e} W")
    print(f"  deltaL={deltaL[0]:.3e}/{deltaL[1]:.3e}/{deltaL[2]:.3e} kg m^2 s^-1, |J1|-|J0|={deltaLnorm:.3e} kg m^2 s^-1, av. dL/dt={deltaL[0]/t_span[1]:.3e}/{deltaL[1]/t_span[1]:.3e}/{deltaL[2]/t_span[1]:.3e} kg m^2 s^-2")
    print(f"  deltaP={deltaP[0]:.3e}/{deltaP[1]:.3e}/{deltaP[2]:.3e} kg m s^-1, |v1|={np.linalg.norm(deltaP)/m:.3e} m/s, reject={np.linalg.norm(deltaD):.3e} m")
    if spin1 != 0: # in case spin1 is not zero
        S1_end = sol.y[6:9, -1]
        S1_precession = np.dot(S1_initial, S1_end)/np.linalg.norm(S1_initial)/np.linalg.norm(S1_end)
        S1_precession = np.arccos(S1_precession)/np.pi*180
        print(f"  S1 precession={S1_precession:.3e} deg, rate={S1_precession/t_span[1]*year:.1e} deg/year")
    if spin2 != 0: # in case spin2 is not zero
        S2_end = sol.y[9:12, -1]
        S2_precession = np.dot(S2_initial, S2_end)/np.linalg.norm(S2_initial)/np.linalg.norm(S2_end)
        S2_precession = np.arccos(S2_precession)/np.pi*180
        print(f"  S2 precession={S2_precession:.3e} deg, rate={S2_precession/t_span[1]*year:.1e} deg/year")
    if analyzeRadiation or analyzePerturbation: # Radiation analysis
        dist_nat = dist/r_scale
        J0_o  = cQ0[1] - (S1_initial + S2_initial)*G/c # initial orbital angular momentum
        J0_o /= np.linalg.norm(J0_o)
        print("Radiation analysis:")
        print(f"  P_orbit={P_orbit:.3e}s")
        print(f"  wavelength={wavelength:.3e}m")
        print(f"  distance={dist/wavelength:.2f} wavelengths")
        #print("  J0_o=", J0_o)
        #print(f"  theta={np.arccos(np.dot(J0_o,N))/np.pi*180}")
        ey = np.cross(J0_o,N)/np.sqrt(1-np.dot(J0_o,N)**2)
        ex = np.cross(ey,N)
        #print("  ex:", ex)
        #print("  ey:", ey)
        #print(f"  ex*N={np.dot(ex,N)}, ey*N={np.dot(ey,N)}, ex*ey={np.dot(ex,ey)}")
        #print(f"  N^N+ex^ex+ey^ey = {np.outer(N,N)+np.outer(ex,ex)+np.outer(ey,ey)}")
        # Getting polarization tensors
        E_plus, E_cross = polarization_tensors(ex, ey)
        #print("  E+ Tensor:\n", E_plus)
        #print("  Ex Tensor:\n", E_cross)
        #print(f"  E+ * E+={np.tensordot(E_plus,E_plus, axes=2)}, Ex * Ex={np.tensordot(E_cross,E_cross, axes=2)}, E+ * Ex={np.tensordot(E_plus,E_cross, axes=2)}")
        h_far  = [project_onto_basis( farField(sol.y[:,k], dist_nat, N), E_plus, E_cross) for k in range(len(sol.y[0]))] # far field polarization h+/hx
        h_near = [project_onto_basis(nearField(sol.y[:,k], dist_nat, N), E_plus, E_cross) for k in range(len(sol.y[0]))] # near field perturbation tensor
    if analyzePerturbation:
        chartX = [t_nat_eval[k]*t_scale for k in range(len(sol.y[0]))] # time values
        #chartY1 = [dist_nat/2/my*h_far[k][0] for k in range(len(sol.y[0]))] # h+ values
        #chartY2 = [dist_nat/2/my*h_far[k][1] for k in range(len(sol.y[0]))] # hx values
        chartY1 = [h_far[k][0] for k in range(len(sol.y[0]))] # h+ far field
        chartY2 = [h_near[k][0] for k in range(len(sol.y[0]))] # h+ near field
        plt.figure(figsize=(10, 8))
        plt.plot(chartX, chartY1, color='red', label='h+')
        plt.plot(chartX, chartY2, color='blue', label='hx')
        plt.xlabel('time (s)')
        plt.ylabel('(D/(2*my)*h')
        plt.title(f"Perturbation direction {N[0]:.2f}/{N[1]:.2f}/{N[2]:.2f}")
        plt.legend()
        plt.grid(True)
        #plt.axis('equal')
        plt.show()

    # HTML graph
    # Find the range and midpoint of data on all three axes
    max_range = np.array([max(r1[0]), max(r1[1]), max(r1[2]), max(r2[0]), max(r2[1]), max(r2[2])]).max()
    min_range = np.array([min(r1[0]), min(r1[1]), min(r1[2]), min(r2[0]), min(r2[1]), min(r2[2])]).min()
    chartL = (max_range-min_range)*0.2
    max_range += chartL
    min_range -= chartL

    def annotationsFunc(k):
        global my
        if my < 1e-100: my = 1
        return [dict( # dynamic annotation linked to slider
                    text=f"|v|={np.linalg.norm(sol.y[3:6,k]):.3e} c, E={cQ[k][0]/normE0:.3f}, dE={cQ[k][2]:.4e}W, J=({cQ[k][1][0]/normJ0:.3f}/{cQ[k][1][1]/normJ0:.3f}/{cQ[k][1][2]/normJ0:.3f}), |J|={np.linalg.norm(cQ[k][1])/normJ0:.3f}<br>S1=({S1[0][k]/normS10:.3f}/{S1[1][k]/normS10:.3f}/{S1[2][k]/normS10:.3f}), |S1|={lenS1[k]:.3f}, S2=({S2[0][k]/normS20:.3f}/{S2[1][k]/normS20:.3f}/{S2[2][k]/normS20:.3f}), |S2|={lenS2[k]:.3f}, h+/hx={1/2/my*h[k][0]:.3e}/{1/2/my*h[k][1]:.3e}",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.5, y=-0.02,
                    xanchor='center', yanchor='top', align='center',
                    font=dict(size=12, color="black"),
                ), dict( # static annotation at top of chart
                    text=f"m1/2={m1:.3e}/{m2:.3e}kg, d={normD0:.3e}m, |S1/2|={normS10*G/c:.3e} / {normS20*G/c:.3e}kg*m^2/s, <br>E0={cQ0[0]:.3e}J, |J0|={normJ0:.3e}kg*m^2/s",
                    xref="paper", yref="paper",
                    x=0.15, y=1.05,  # Position the subtitle below the main title
                    xanchor='left', yanchor='top', align='center',
                    showarrow=False,
                    font=dict(size=12, color="black"),
                )
            ]
    menues = [{
                "type": "buttons",
                "buttons": [
                    {
                        "label": "Play",
                        "method": "animate",
                        "args": [None, {"frame": {"duration": 0, "redraw": True}, "fromcurrent": True}]
                    },
                    {
                        "label": "XY Plane",
                        "method":"relayout",
                        "args": [{"scene.camera.eye": {"x": 0, "y": 0, "z": 1},
                               "scene.camera.up": {"x": 0, "y": 1, "z": 0},
                               "scene.camera.projection.type": "orthographic",
                               "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "XZ Plane",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 0, "y": -1, "z": 0},
                               "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                               "scene.camera.projection.type": "orthographic",
                               "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "YZ Plane",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 1, "y": 0, "z": 0},
                               "scene.camera.projection.type": "orthographic",
                               "scene.aspectratio": {"x": 1.8, "y": 1.8, "z": 1.8}
                        }]
                    },
                    {
                        "label": "3D View",
                        "method": "relayout",
                        "args": [{"scene.camera.eye": {"x": 0.5, "y": -0.5, "z": 1.2},
                               "scene.camera.up": {"x": 0, "y": 0, "z": -1},
                               "scene.camera.projection.type": "perspective",
                               "scene.aspectratio": {"x": 1.2, "y": 1.2, "z": 1.2}
                        }]
                    }
                ],
            }]

    if plotOrbit: # plot orbit
        if scientific: # Show grids, scales and exis
            orbitScene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                xaxis=dict(range=[min_range, max_range]),
                yaxis=dict(range=[min_range, max_range]),
                zaxis=dict(range=[min_range, max_range]),
                aspectmode='cube',
                camera=dict(eye = dict(x=0.5, y=-0.5, z=1.1), up  = dict(x=0, y=0, z=-1)),
                aspectratio=dict(x=1.2, y=1.2, z=1.2),
            )
        else: # Hide grids, scales and exis
            orbitScene=dict(
                xaxis=dict(range=[min_range, max_range], title_text='', showbackground=False, showticklabels=False, zeroline=False),
                yaxis=dict(range=[min_range, max_range], title_text='', showbackground=False, showticklabels=False, zeroline=False),
                zaxis=dict(range=[min_range, max_range], title_text='', autorange=False, showbackground=False, showticklabels=False, zeroline=False),
                xaxis_showgrid=False, yaxis_showgrid=False, zaxis_showgrid=False,
                xaxis_showline=False, yaxis_showline=False, zaxis_showline=False,
                aspectmode='cube',
                camera=dict(eye = dict(x=0.5, y=-0.5, z=1.1), up  = dict(x=0, y=0, z=-1)),
                aspectratio=dict(x=1.2, y=1.2, z=1.2),
            )

        fig = go.Figure(
            data=[ # Create a Plotly figure
                go.Scatter3d(x=[r1[0][0]], y=[r1[1][0]], z=[r1[2][0]], mode='markers', name = 'Body 1', marker=dict(size=10, color='red')),
                go.Scatter3d(x=[r2[0][0]], y=[r2[1][0]], z=[r2[2][0]], mode='markers', name = 'Body 2', marker=dict(size=10, color='blue')),
                go.Scatter3d(x=[0, cQ[0][1][0]/normJ0*chartL], y=[0, cQ[0][1][1]/normJ0*chartL], z=[0, cQ[0][1][2]/normJ0*chartL], mode='lines', name='Angular Impulse', line=dict(color='black', width=4)),
                go.Scatter3d(x=[r1[0][0], r1[0][0]+S1[0][0]/normS10*chartL], y=[r1[1][0], r1[1][0]+S1[1][0]/normS10*chartL], z=[r1[2][0], r1[2][0]+S1[2][0]/normS10*chartL], mode='lines', name='S1', line=dict(color='red', width=4)),
                go.Scatter3d(x=[r2[0][0], r2[0][0]+S2[0][0]/normS20*chartL], y=[r2[1][0], r2[1][0]+S2[1][0]/normS20*chartL], z=[r2[2][0], r2[2][0]+S2[2][0]/normS20*chartL], mode='lines', name='S2', line=dict(color='blue', width=4)),
                go.Scatter3d(x=[0], y=[0], z=[0], mode='markers', name='Origin', marker=dict(size=4, color='black')),
                go.Scatter3d(x=r1[0], y=r1[1], z=r1[2], mode='lines', name = 'Trj 1', line=dict(color='red', width=1)),
                go.Scatter3d(x=r2[0], y=r2[1], z=r2[2], mode='lines', name = 'Trj 2', line=dict(color='blue', width=1)),
            ],
            layout=go.Layout(
                title = name,
                scene=orbitScene,
                annotations = annotationsFunc(0),
                updatemenus=menues,
                sliders=[{
                    "steps": [
                        {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                         "label": f"{i*dt:.3e}s",
                         "method": "animate"
                         } for i in range(0,nn-1,10)
                    ],
                }],
            ),
            frames=[go.Frame(
                data=[
                    go.Scatter3d(x=[r1[0][k+1]], y=[r1[1][k+1]], z=[r1[2][k+1]], mode='markers', name = 'Body 1', marker=dict(size=10, color='red')),
                    go.Scatter3d(x=[r2[0][k+1]], y=[r2[1][k+1]], z=[r2[2][k+1]], mode='markers', name = 'Body 2', marker=dict(size=10, color='blue')),
                    go.Scatter3d(x=[0, cQ[k+1][1][0]/normJ0*chartL], y=[0, cQ[k+1][1][1]/normJ0*chartL], z=[0, cQ[k+1][1][2]/normJ0*chartL], mode='lines', name='Angular Impulse', line=dict(color='black', width=4)),
                    go.Scatter3d(x=[r1[0][k+1], r1[0][k+1]+S1[0][k+1]/normS10*chartL], y=[r1[1][k+1], r1[1][k+1]+S1[1][k+1]/normS10*chartL], z=[r1[2][k+1], r1[2][k+1]+S1[2][k+1]/normS10*chartL], mode='lines', name='S1', line=dict(color='red', width=4)),
                    go.Scatter3d(x=[r2[0][k+1], r2[0][k+1]+S2[0][k+1]/normS10*chartL], y=[r2[1][k+1], r2[1][k+1]+S2[1][k+1]/normS10*chartL], z=[r2[2][k+1], r2[2][k+1]+S2[2][k+1]/normS10*chartL], mode='lines', name='S2', line=dict(color='blue', width=4))
                ],
                layout=go.Layout(annotations=annotationsFunc(k)),
                name=f"frame{k}"
            ) for k in range(0,nn-1, 10)]
        )
        fig.update_layout(autosize=True)  # Disable autosizing to use specified size
        fig.show()
        #fig.write_html('2-body-orbit-animated.html') # Save to HTML

    if analyzeRadiation: # Radiation analysis
        GW_RESOLUTION = 60 #45
        GW_TIMESAMPLE = 5  #5
        colorscale='Blackbody'
        #colorscale='RdBu'

        # Trajectories for two bodies
        x1 = [r1[0][k]*r_scale for k in range(0,nn-1,GW_TIMESAMPLE)]
        y1 = [r1[1][k]*r_scale for k in range(0,nn-1,GW_TIMESAMPLE)]
        x2 = [r2[0][k]*r_scale for k in range(0,nn-1,GW_TIMESAMPLE)]
        y2 = [r2[1][k]*r_scale for k in range(0,nn-1,GW_TIMESAMPLE)]
        r12 = [distance(sol.y[:,k]) for k in range(0,nn-1,GW_TIMESAMPLE)] # body distance
        nnn=len(x1)
        print(f"  Resolution: nnn={nnn} timesteps, dt={dt*GW_TIMESAMPLE:.3e}, dt_ret={dist/GW_RESOLUTION/c:.3e}")

        # create the h+ mesh
        xx, yy, times = np.meshgrid(np.linspace(dist/GW_RESOLUTION, dist, GW_RESOLUTION), np.linspace(0, 2*np.pi, GW_RESOLUTION), np.linspace(0, nnn, nnn))
        zz = xx * yy * times # create empty data array
        #zz2 = xx * yy * times # same for near field
        h_max = h_min = 0 # initialize
        k0 = 0 # start index for animation
        print("  Calculating wave", end='')
        for j in range(GW_RESOLUTION): # iterate yy (azimuth)
            print(".", end='')
            azimuth = yy[j,0,0]
            N = np.array([np.cos(azimuth), np.sin(azimuth), 0]) # define wave direction
            ey = np.cross(J0_o,N)/np.sqrt(1-np.dot(J0_o,N)**2)
            ex = np.cross(ey,N)
            E_plus, E_cross = polarization_tensors(ex, ey) # calculate polarization tensors
            h_far  = [project_onto_basis( farField(sol.y[:,k], 1, N), E_plus, E_cross) for k in range(0,nn-1,GW_TIMESAMPLE)] # far field polarization h+/hx
            for i in range(GW_RESOLUTION): # iterate xx (distance)
                dist_obs = xx[j,i,0] # distance to observer
                #dist_nat = dist_obs/r_scale # set observer distance
                #h_near = [project_onto_basis(nearField(sol.y[:,k], dist_nat, N), E_plus, E_cross) for k in range(0,nn-1,GW_TIMESAMPLE)] # far field polarization h+/hx
                for k in range(nnn): # iterate times
                    t_ret = dist_obs/c/GW_TIMESAMPLE/dt # time retardation
                    k_floor = int(np.floor(k - t_ret))
                    k_ceil = int(np.ceil(k - t_ret))
                    qq = (k - t_ret) - k_floor # interpolation weight
                    h_plus = (1-qq)*h_far[k_floor][0] + qq*h_far[k_ceil][0] # interpolate h+
                    h_plus = h_plus/2/my/np.log(np.sqrt( dist_obs**2+r12[k]**2) ) # apply scale
                    #h_plus_nf = (1-qq)*h_near[k_floor][0] + qq*h_near[k_ceil][0] # near field
                    #h_plus_nf = h_plus_nf/2/my * (dist_obs/np.log(dist_obs)) # near field
                    h_max = max(h_plus, h_max)
                    h_min = min(h_plus, h_min)
                    if k_floor < 0:
                        zz[j,i,k] = 0
                        k0 = k
                        #zz2[j,i,k] = 0 # near field
                    else:
                        zz[j,i,k] = np.round(h_plus, 4) # populate data array
                        #zz2[j,i,k] = np.round(h_plus_nf, 4) # near field

        # Create the figure
        h_scale = max(abs(h_max), abs(h_min))
        h_min = -h_scale
        h_max = h_scale
        print()
        print(f"  h_scale={h_scale:.3e}, h_min={h_min:.3e}, h_max={h_max:.3e}")
        print( "  Producing chart")
        z1 = np.zeros_like(x1)  # put the 2 bodies above the GW
        z2 = np.zeros_like(x1)
        xxx = np.round(xx[:,:,0]*np.cos(yy[:,:,0]), 3)
        yyy = np.round(xx[:,:,0]*np.sin(yy[:,:,0]), 3)
        if scientific: # Show grids, scales and exis
            waveScene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                xaxis=dict(range=[-dist, dist]),
                yaxis=dict(range=[-dist, dist]),
                zaxis=dict(range=[h_min, h_max], autorange=False),
                aspectmode='cube',
                camera=dict(eye = dict(x=0.5, y=-0.5, z=1.1), up  = dict(x=0, y=0, z=-1)),
                aspectratio=dict(x=1.2, y=1.2, z=1.2),
            )
        else: # Hide grids, scales and exis
            waveScene=dict(
                xaxis=dict(range=[-dist, dist], title_text='', showbackground=False, showticklabels=False, zeroline=False),
                yaxis=dict(range=[-dist, dist], title_text='', showbackground=False, showticklabels=False, zeroline=False),
                zaxis=dict(range=[h_min, h_max], title_text='', autorange=False, showbackground=False, showticklabels=False, zeroline=False),
                xaxis_showgrid=False, yaxis_showgrid=False, zaxis_showgrid=False,
                xaxis_showline=False, yaxis_showline=False, zaxis_showline=False,
                aspectmode='cube',
                camera=dict(eye = dict(x=0.5, y=-0.5, z=1.1), up  = dict(x=0, y=0, z=-1)),
                aspectratio=dict(x=1.2, y=1.2, z=1.2),
            )

        fig = go.Figure(
            data=[
                go.Scatter3d(x=[x1[k0]], y=[y1[k0]], z=[z1[k0]], mode='markers', marker=dict(color='red'), showlegend = False),
                go.Scatter3d(x=[x2[k0]], y=[y2[k0]], z=[z2[k0]], mode='markers', marker=dict(color='blue'), showlegend = False),
                go.Surface(z=zz[:,:,k0], x=xxx, y=yyy, colorscale=colorscale, cmin=h_min, cmax=h_max),
                #go.Surface(z=zz2[:,:,k0], x=xxx, y=yyy, colorscale='RdBu', cmin=h_min, cmax=h_max)
            ],
            layout=go.Layout(
                title='Binary System with GW [D/(2*my*log(D))*h+]',
                scene=waveScene,
                # Optional: hide the plot background
                #plot_bgcolor='rgba(0,0,0,0)', # Make plot background fully transparent
                #paper_bgcolor='rgba(0,0,0,0)', # Make paper background fully transparent
                updatemenus=menues,
                sliders=[{
                    "steps": [
                        {"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                         "label": f"{i*GW_TIMESAMPLE*dt:.3e}s",
                         "method": "animate"
                         } for i in range(k0, nnn-1, 1)
                    ],
                }],
            ),
            frames=[
                go.Frame(
                    data=[
                        go.Scatter3d(x=[x1[k]], y=[y1[k]], z=[z1[k]], mode='markers', marker=dict(color='red', size=5)),
                        go.Scatter3d(x=[x2[k]], y=[y2[k]], z=[z2[k]], mode='markers', marker=dict(color='blue', size=5)),
                        go.Surface(z=zz[:,:,k], x=xxx, y=yyy, colorscale=colorscale, cmin=h_min, cmax=h_max),
                        #go.Surface(z=zz2[:,:,k], x=xxx, y=yyy, colorscale='RdBu', cmin=h_min, cmax=h_max)
                    ],
                    name=f"frame{k}"
                )
                for k in range(k0, nnn-1, 1)
            ]
        )
        #fig.update_layout(scene=dict(zaxis=dict(range=[-1, 1], nticks=4)))
        fig.show()
        fig.write_html('2-body-GW-animated.html') # Save to HTML

    print()
    return

__Earth orbit around Sun__  
This simulation is primarily for testing purposes. It accurately reproduces the earths orbit around the sun, using the Post Newtonian Approximation. The only interesting result is the minimal GW radiation of 200 W (matching the power of a light bulb). This effect is not present in neither the Newton nor the GR 1-body models.

In [ ]:
initialize_globals()
name ="Earth orbit around Sun"
m1 = m_sun  # Mass of the Sun, kg
m2 = 5.972e24  # Mass of the Earth, kg
r0 = 1.496e11
r_initial = np.array([r0, 0, 0])  # Earth at 1 AU
v_initial = np.array([0, np.sqrt(G * m1 / np.linalg.norm(r_initial)), 0])  # Earth circular orbit velocity
t_span = (0, 10*year)  # One year in seconds
simulate_orbit()

__Mercury orbit around Sun__  
Mercury's orbit shows a perihelion shift of 574 arcsec per Julian century. 532 arcsec are explained by the tugs of other solar bodies (ie planets). This anomalous rate of precession (43 arcsec) of the perihelion of Mercury's orbit was first recognized in 1859 as a problem in celestial mechanics, by Urbain Le Verrier. A number of ad hoc and ultimately unsuccessful solutions (such as an additional planet "Vulcan" in the solar system) were proposed, but they tended to introduce more problems.  
Einsteins theory of General Relativity precisely explains the 43 arcsec. Einsteins 1915 paper already included the quantification of this effect and was seen as an important confirmation of this new theory.  
The simulation leads to precisely this effect (0.1 arcsec per Mercury orbit around the sun, times 400 orbits per Julian century).

In [ ]:
initialize_globals()
name = "Mercury orbit around Sun"
m1 = m_sun
m2 = 3.301e23
r0 = 4.601e10 # periastron distance
r_initial = np.array([r0, 0, 0])
v_initial = np.array([0, 58980.0, 0])  # periastron velocity
t_span = (0, 10*year)  #ca 40 orbits
analyzePeriastron = True # shift should be 0.1 arsec per orbit
rtol = 2.3e-14 # precision needed to recognize change in orbital period
simulate_orbit()

__S2 orbit around Sagittarius A*__  
Sagittarius A* (SgtA*) is the central BH in our galaxy, the milkyway. Its mass is 4.2 million sun masses, and its Schwarzschild radius is approx. 0.1 au. S2 is a star orbiting SgtA* in a highy eccentric orbit. Its mass is 15 sun masses, and the periastron distance is 17 light hours (120 au). The orbital period is 17 years. S2 reaches 2.56% the speed of light at its periastron.  
The simulation correctly captures the periastron shift of 713 arc seconds per orbit (this result is also achievable by the 1-body approach). It also provides the GW radiation power of 2e23 W (roughly one per mille of the sun's luminosity of 3.8e26 W) and the orbital decay of approx. 1e-4 s/year. Both these values are de minimis and can not be measured.  
Historical context: SgtA*, only 27'000 light years away from earth, has been identified as a supermassive BH in the 1980ies, by the work of Reinhard Genzel and Andrea Ghez who were awared the Nobel Price for their discoveries in 2020. SgtA* is not directly visible. It has been identified by orbiting objects such as its accretion disk and several stars (called the S group), amongst it S2. A picture of SgtA* accretion disk has been published in 2022.  
  
https://cdn.eso.org/images/original/eso2208-eht-mwa.tif

In [ ]:
initialize_globals()
name = "S2 orbit around Sagittarius A*"
m1 = 4.211e6 * m_sun  # ca. 4.2 million times Mass of the Sun, kg
m2 = 15 * m_sun  # 15 sun masses
r0 = 1.8e13 # S2 periastron (17 light hours)
spin1 = 0.95 # give SgtA* a spin
S1n0 = np.array([0, 1, 0]) # spin is along y axis
r_initial = np.array([r0, 0, 0])  # S2 periastron at 17 light hours
v_initial = np.array([0, 7.65e6, 0])  # S2 periastron velocity
t_span = (0, 150*year)  # period in years
analyzePeriastron = True # should be 713 arsec per orbit
rtol = 2.3e-14 # precision needed to recognize change in orbital period
simulate_orbit()

__Inspiralling binaries__  
This is a hypothetical constellation (developed by JO) to demonstrate the GW radiation. It is a binary BH system with 30 and 20 sun masses on a quasi circular orbit at an initial distance of some 10 Schwarzschild radii. Both BH have very high spins of 0.9, pointing in different directions, hence the orbit is highly precessing due to the high spins involved. The GW radiation is large due to the similar sizes of the two masses involved. The simulation covers the final 30 orbits up to the collapse. It is an extreme constellation to demonstrate all GR effects at play. The constellation cannot be solved with the 1-body approach.

In [ ]:
initialize_globals()
name = "Inspiralling binaries"
m1 = 30 * m_sun
m2 = 20 * m_sun
spin1 = 0.9 # dimensionless spin parameter [-1..+1]
spin2 = 0.9 # dimensionless spin parameter [-1..+1]
S1n0 = np.array([0, 1, 0]) # spin along y axis as initial orbit is in equatorial plane
S2n0 = np.array([1, 0, 0]) # spin along z axis
r0 = 1475e3 # ca 10 Schwarzschild radii
v0 = 0.2236*c
r_a = G*(m1+m2)*r0 / (2*G*(m1+m2) - v0**2*r0)
P_orbit = 2*np.pi*np.sqrt( r_a**3/G/(m1+m2) )
r_initial = np.array([r0, 0, 0])
v_initial = np.array([0, v0, 0])
t_span = (0, 40 * P_orbit)  # period in s
analyzePeriastron = True
analyzeRadiation = True
N = np.array([1, 0, 0]) # direction of observer
wavelength = 0.5*c*P_orbit # observer distance
dist = 2*wavelength
simulate_orbit()

__Hulse-Taylor Pulsar__ (known as PSR B1913+16, PSR J1915+1606 or PSR 1913+16)  
The Hulse–Taylor pulsar  is a binary star system, some 21'000 light years away from earth (in the Milkyway), composed of a neutron star and a pulsar which orbit around their common center of mass. It is the first binary pulsar ever discovered.  
The pulsar and its neutron star companion both follow elliptical orbits around their common center of mass. The period of the orbital motion is 7.75 hours, and the two neutron stars are believed to be nearly equal in mass, about 1.4 solar masses. Radio emissions have been detected from only one of the two neutron stars. The minimum separation at periastron is about 1.1 solar radii; the maximum separation at apastron is 4.8 solar radii. The orientation of periastron changes by about 4.2 degrees per year in direction of the orbital motion (relativistic precession of periastron).  
The simulation correctly captures the periastron shift (13.5 arc sec per orbit) and the GW radiation (7.35e24 W). With this comparatively large energy loss due to gravitational radiation, the rate of decrease of orbital period is 76.5 microseconds per year, the rate of decrease of semimajor axis is 3.5 meters per year, and the calculated lifetime to final inspiral is 300 million years. The orbit reaches a relative acceleration of some 68 g, which is divided roughly equally between both companions when measured against the common COM. The 2-body approach is required due to the similar size of the two masses.  
Historical context: The pulsar was discovered by Russell Alan Hulse and Joseph Hooton Taylor Jr., of the University of Massachusetts Amherst in 1974. Their discovery of the system and analysis of it earned them the 1993 Nobel Prize in Physics for the discovery of a new type of pulsar, a discovery that has opened up new possibilities for the study of gravitation. The pulsar is identified based on its "radio beam" sweeping space, creating a radio pulse measurebale on the earth once every rotation (like a light house). The pulse frequency is 59 ms (17 rotations per second). The system is not visible in the optical spectrum (neutron star diameter is around 10km, with low surface luminosity).     
Source: https://en.wikipedia.org/wiki/Hulse–Taylor_pulsar  
Paper: https://arxiv.org/pdf/1411.3930

In [ ]:
initialize_globals()
name = "Hulse-Taylor Pulsar"
m1 = 1.441 * m_sun
m2 = 1.387 * m_sun
r_m1 = 10000 # radius of the rotating mass, m
T_m1 = 0.059 # time per revolution, s
I_m1 = 2/5*m1*r_m1**2 # Moment of Inertia, kg m^2
omega = 2*np.pi/T_m1 # s^-1
spin1 = c*omega*I_m1/G/m1**2 # 1
print(f"Spin parameter={spin1:.3e}")
r0 = 746.6e6 # periastron distance
r_initial = np.array([r0, 0, 0])
v_initial = (1+m2/m1)*np.array([0, 459.4e3, 0])  # periastron velocity
t_span = (0, 20*7.7519*3600)  #7.7510 hours per orbit
analyzePeriastron = True # shift should be 13 arsec per orbit
rtol = 2.3e-14 # precision needed to recognize change in orbital period

simulate_orbit()

__PSR J1757-1854__  
PSR J1757–1854 is a remarkable binary pulsar system located within our own Milky Way galaxy, roughly 25,000 light-years away in the constellation Sagittarius. It consists of two neutron stars, one of which is a recycled millisecond pulsar with a mass of about 1.34 solar masses, and the other a more slowly rotating companion weighing approximately 1.40 solar masses. These two ultra-dense remnants orbit each other in a highly eccentric path every 4.4 hours, making the system one of the most relativistic known in our galaxy. Several strong-field relativistic effects predicted by general relativity are clearly observed in this system. Among them is a rapid advance of periastron—the point of closest orbital approach—which proceeds at a remarkable rate of about 10.3651 degrees per year. This is one of the fastest such rates ever measured in a binary pulsar. In addition, the orbit is slowly shrinking due to the emission of gravitational waves, with the orbital period decaying at a rate of approximately 167 microseconds per year, leaving a calculated time to inspiral (coalescence) of some 76 million years. This decay matches the predictions of general relativity to an extraordinary degree of precision, reinforcing the theory’s accuracy under extreme conditions. Other relativistic phenomena also play a role, such as Shapiro delay, where the pulsar’s radio pulses are delayed as they pass through the gravitational field of the companion, and gravitational time dilation, where the pulsar’s timing is affected by variations in orbital velocity and gravitational potential. Together, these effects make PSR J1757–1854 a natural laboratory for testing Einstein’s theory in regimes inaccessible on Earth.  
The simulation matches the periastron advance (10.3 deg/year) and orbital decay (167 ms/year). It also provides a GW radiation of 4.3e+25 W which is some 6 times larger than that of the Hulse-Taylor pulsar (despite the similar total mass of both systems). The orbit reaches a relative acceleration of some 137 g, which is divided roughly equally between both companions when measured against the common COM.  
https://arxiv.org/pdf/2305.14733

In [ ]:
initialize_globals()
name = "PSR J1757 1854"
# Simulation closely matches know values for periastron shift (18.8 arcsec/year)
# and orbital period change (-166us/year)
m1 = 1.3406 * m_sun # Pulsar
m2 = 1.3922 * m_sun # Companion
T_m1 = 0.0215 # Spin period, s
T_orbit = 0.1835378358*24*3600 # Orbital period, s (from days)
inclination = 85/180*np.pi
e = 0.6058171 # eccentricity, 1
I_m1 = 1.2e45/1000/10000 # Moment of Inertia, kg m^2 (from g cm^2)
omega = 2*np.pi/T_m1 # spin angular velocity, rad s^-1
spin1 = c*omega*I_m1/G/m1**2 # 1
spinAngle = 52/180*np.pi # misalignment between spin and orbital angular momentum, deg
S1n0 = np.array([np.sin(spinAngle), 0, np.cos(spinAngle)]) # spin direction
print(f"Spin parameter={spin1:.3e}")
r_sun = 6.96342e8 # m
r_a = 1.89*r_sun # semi major axis, m
r0 = r_a*(1-e) # periastron, m
r_initial = np.array([r0, 0, 0])
v_initial = np.array([0, np.sqrt(G*(m1+m2)*(2/r0 - 1/r_a)), 0]) # periastron velocity
t_span = (0, 20*T_orbit)  # number of orbits to be simulated
analyzePeriastron = True # shift should be 18.8 arcsec/orbit (10.365 deg in 1 year over 1985 full orbits)
rtol = 2.3e-14 # precision needed to recognize change in orbital period
simulate_orbit()

__OJ 287__  
This blazar is 4 billion lightyears from earth. Its quasi periodic outbursts have been photographed for approximately 120 years. It is a super massive black hole binary (SMBHB). The primary BH mass is approx. 18.3 billion sun masses, the secondary BH is approx. 150 million sun masses. The quasi periodic outbursts stem from the secondary BH crossing the accretion disk of the primary BH twice in one orbit. The orbital period is currently approx. 12 years with a local relative periastron velocity of 0.27*c. The periastron distance (3'250 au) is approx. 9 times the Schwarzschild radius of the primary BH (361 au). The Schwarzschild radius of the secondary BH is approx. 3 au. The system is expected to merge in approx. 10'000 years. The periastron shift is ca. 39 degrees per orbit. The GR effects in OJ 287 are most extreme.  
The simulation results in an orbital period of 12 years (correct), a periastron shift of 39 degrees per orbit (correct), an orbital period decay of -36'000 seconds per year  (1e-3 decay rate) (correct), and a GW radiation of -3e41 W. The orbital period reduces by 5 days each orbit. The orbital precession due to Spin-Orbit coupling is clearly visible in the simulation: The chart indicates an orbital precession of some 0.15 deg/year, or 1.8 deg per full orbit (ChatGPT says according to Dey et al. 2018, Valtonen et al. the orbital plane precesses at a rate of approximately 0.5 deg/year - not verified)  
Calibration: m1, m2, r0 , spin1 taken from Wikipedia. e chosen to match orbital period (12 years)  
Tip: Running the simulation over 10'000 years gets close to inspiral. Simulation takes ca. 8 mins.  
  
Illustration: https://upload.wikimedia.org/wikipedia/commons/9/92/BlackHoleDiskFlareInGalaxyOJ287-animation-20200428.webm  
Paper: https://arxiv.org/pdf/1808.09309

In [ ]:
initialize_globals()
name = "OJ 287"
au = 149597870700 # in meter
m1 = 1.83e10 * m_sun  # ca. 18 billion times Mass of the Sun, kg
m2 = 1.5e8 * m_sun  # 150 million sun masses
r0 = 3250 * au # periastron distance in meter
e = 0.64 # eccentricity
r_a = r0 / (1-e) # semi major axis
v_local = np.sqrt(G*(m1+m2)*(2/r0 - 1/r_a))
lorenz = 1 - (v_local/c)**2
spin1 = 0.38 # give a spin
S1n0 = np.array([0, 1, 0]) # spin is along z axis
r_initial = np.array([r0, 0, 0])  # periastron vector
v_initial = np.array([0, lorenz*v_local, 0])  # periastron velocity
t_span = (0, 150*year)  # period in years
analyzePeriastron = True # should be 39 deg per orbit
rtol = 2.3e-14 # precision needed to recognize change in orbital period
simulate_orbit()

__Prograde particle orbit around rotating black hole__ (yukterez.net)  
This constellation is to test the orbit solver with a known result. The results match to a large degree. The constellation does not require the 2-body approach (1-body is sufficient) as the mass ratio is 1e5.  
https://upload.wikimedia.org/wikipedia/commons/6/6b/Orbit_um_ein_rotierendes_schwarzes_Loch_%28Animation%29.gif

In [ ]:
initialize_globals()
name = "Prograde particle orbit around rotating black hole (yukterez.net)"
# E=0.9352, Lz=2.3718, Q=3.8251
G = c = year = 1
rs = 2
m1 = 1
m2 = 0.00001
spin1 = 0.90 # dimensionless spin parameter [-1..+1]
spin2 = 0 # dimensionless spin parameter [-1..+1]
r0 = 7.0  # yukterez.net
x0 = np.sqrt(r0**2 + spin1**2)
theta0 = 0
vtheta0 = 0.256074/r0 # local velocity of 0.4, i0 = arctan(5/6)
vphi0 = 0.307289/x0 # note x0 here (but not for vtheta)
vphi0 += omegaZAMO(r0, x0, rs, spin1, theta0, vtheta0, vphi0) # adjust local velocity with omega

factor = 0.705  #
vz0 = -vtheta0 * r0 * factor # local velocity of 0.5
vy0 = +vphi0 * x0 * factor # note x0 here (but not for vtheta)
r_initial = np.array([x0, 0, 0])
v_initial = np.array([0, vy0,  vz0])
t_span = (0, 953/factor)  # period in s
analyzePeriastron = True

simulate_orbit()

__Retrograde particle orbit around rotating black hole__ (yukterez.net)  
This constellation is to test the orbit solver with a known result. The results match to a large degree. The constellation does not require the 2-body approach (1-body is sufficient) as the mass ratio is 1e5.  
https://upload.wikimedia.org/wikipedia/commons/7/74/Orbit_around_a_rotating_Kerr_black_hole.gif

In [ ]:
initialize_globals()
name = "Retrograde particle orbit around rotating black hole (yukterez.net)"
# E=0.9565, Lz=-0.8303, Q=13.413
G = c = year = 1
rs = 2
m1 = 1
m2 = 0.00001
spin1 = 0.95 # dimensionless spin parameter [-1..+1]
spin2 = 0 # dimensionless spin parameter [-1..+1]
r0 = 6.5  # yukterez.net
x0 = np.sqrt(r0**2 + spin1**2)
theta0 = 0
vtheta0 = -np.cos(11/50)/2/r0 # local velocity of 0.5
vphi0 = -np.sin(11/50)/2/x0 # note x0 here (but not for vtheta)
vphi0 += omegaZAMO(r0, x0, rs, spin1, theta0, vtheta0, vphi0) # adjust local velocity with omega

factor = 0.700  #spin=95, approx= 2.5: use factor = 0.7
vz0 = -vtheta0 * r0 * factor # local velocity of 0.5
vy0 = +vphi0 * x0 * factor # note x0 here (but not for vtheta)
r_initial = np.array([x0, 0, 0])
v_initial = np.array([0, vy0, vz0])
t_span = (0, 1640/factor)  # period in s
analyzePeriastron = True

simulate_orbit()

__Photon passing by the sun near its surface__  
General Relativity predicts that a light ray, passing by the surface of the sun hitting the earth, should be deflected by 1.7 arcsec. This number was first correctly determined bei Einstein in 1915. The first observation of light deflection was performed by noting the change in position of stars as they passed near the Sun on the celestial sphere. The observations were performed by Arthur Eddington and his collaborators (see Eddington experiment) during the total solar eclipse of May 29, 1919, when the stars near the Sun (at that time in the constellation Taurus) could be observed. The result was considered spectacular news and made the front page of most major newspapers. It made Einstein and his theory of general relativity world-famous.  
The simulation leads to the correct deflection of 1.7 arcsec.

In [ ]:
initialize_globals()
name ="Photon passing by the sun near its surface "
m1 = m_sun  # Mass of the Sun, kg
m2 = 0  # Test mass limit case
r0 = 1.496e11 # Earth distance from sun
r_initial = np.array([r0, 0, 0])  # Earth at 1 AU
vy0 = 0.0047*c # calibrated such that photon targets Sun radius (ca. 7e8 m)
v_initial = np.array([-np.sqrt(c**2-vy0**2), vy0, 0])  # light speed
t_span = (0, 16*60)  # One minute
analyzeDeflection = True
simulate_orbit()

__Photon bending around a spinning black hole__  
This is a hypothetical constellation, resulting in a highly deflected light ray

In [ ]:
initialize_globals()
G = c = m1 = year = 1 # natural units
m2 = 0  # Test mass limit case
name = "Photon bending around a spinning black hole"
spin1 = 0.5 #
r0 = 58  # Starting point near the black hole
r_initial = np.array([r0, 0, 0])  #
v_initial = np.array([-0.9, 0, 0.09])
v_initial /= np.linalg.norm(v_initial) # light speed
t_span = (0, 140*year)  #
simulate_orbit()

__Photon testing__  
This is a test bed to explore different constellations of light bending

In [ ]:
initialize_globals()
G = c = m1 = year = 1 # natural units
m2 = 0  # Test mass limit case
name = "Photon testing"
spin1 = 1 #
r0 = 58  # Starting point near the black hole
r_initial = np.array([r0, 0, 0])  #
v_initial = np.array([-0.9, 0, -0.2])
v_initial /= np.linalg.norm(v_initial) # light speed
t_span = (0, 140*year)  #
analyzePeriastron = True
analyzeDeflection = True
simulate_orbit()

__Kidder Fig 4__  
This constellation resembles a hypothetical constellation analysed in the paper (see Fig.4)  
Coalescing binbary systems of compact objects to (post)^5/2-Newtonian order. V. Spin Effects  
Lawrence E. Kidder et al  
From this paper, most of the formulas for the post Newtonian approximation used in this simulation have been sourced.  
https://arxiv.org/pdf/gr-qc/9506022

In [ ]:
initialize_globals()
name = "Kidder Fig 4"
G = c = year = 1 # natural units
m1 = 10 # Pulsar
m2 = 1.4 # Companion
e = 0.0 # eccentricity, 1
spin1 = 1 # maximal rotation
spin2 = 1 #
spin1Angle = 30/180*np.pi # misalignment between spin and orbital angular momentum, deg
S1n0 = np.array([np.sin(spin1Angle), 0, np.cos(spin1Angle)]) # spin direction
r_a = 15*(m1+m2) # begin inspiral at 15m
r0 = r_a*(1-e) # periastron
v0 = v_circular(np.array([0, 0, 1])) # determine v0 assumimng circular orbit, providing direction of initial orbital angular momentum
P_orbit = 2*np.pi*np.sqrt( r_a**3/G/(m1+m2) )
r_initial = np.array([r0, 0, 0])
v_initial = np.array([0, v0, 0]) # periastron velocity
t_span = (0, 20*6.28*r_a/v0)  # number of orbits to be simulated
r_terminal = 10 # terminate at 10m
rtol = 2.3e-14 # precision needed
plotOrbit = False
analyzePeriastron = True
analyzePerturbation = True
analyzeRadiation = False
N = np.array([1, 0, 0]) # direction of observer
wavelength = 0.5*c*P_orbit # observer distance
dist = 2*wavelength
simulate_orbit()